# Traveling Salesman Problem

We are given a [complete graph](https://en.wikipedia.org/wiki/Complete_graph) which means that every pair of distinct vertices is connected by a unique edge. Each edge has traveling cost associated with it. We need to find a shortest route that goes throught all the nodes and comes back to the original node.

## Representation

We can represent traveling costs as a simple matrix.

In [1]:
i = float('inf')
t =  [
    [i, 3, 1, 8],
    [3, i, 4, 4],
    [1, 4, i, 7],
    [8, 4, 7, i],
]

This represents 4 nodes (0 through 3) where cost of traveling from 0 to 3 is

In [2]:
t[0][3]

8

In [3]:
t[1][1]

inf

## Brute force solution

In [95]:
from itertools import permutations

def total_cost(matrix, route, loop=True):
    # we are going from node a to node b in each step
    a = list(route)
    b = a[1:] + [a[0]]

    # calculate the route cost
    cost = 0
    for ii, jj in zip(a, b):
        cost += matrix[ii][jj]
    if not loop:
        # exclude path back to start
        cost -= matrix[route[0]][route[-1]]
    return cost


def brute_force(matrix, start=0, stop=None, loop=True):
    if not stop:
        stop = len(matrix)
    current_min = float('inf')
    current_route = []


    for route in permutations(range(start, stop)):
        route_cost = total_cost(matrix, route, loop)
        # check if we got new min route
        if route_cost < current_min:
            current_min = route_cost
            current_route = route
    
    return list(current_route), current_min

## Greedy solution
What if we just always pick the shortest distance from all available routes in a current node. We start with first node and pick the cheapest node that we haven't picked yet.

In [5]:
def get_min_node(nodes, visited_set):
    min_cost = pow(10, 10)
    node_position = None
    for position, cost in enumerate(nodes):
        # make sure we haven't visited the node yet, we are not trying to go to
        # the same node, and current cost is the minimum that we've seen so far
        if position in visited_set or isinstance(cost, str) or cost >= min_cost:
            continue

        min_cost = cost
        node_position = position

    return node_position

def greedy_recursion(matrix, route):
    current_node = route[-1]
    min_node = get_min_node(matrix[current_node], set(route))

    if min_node is None:
        return route

    return greedy_recursion(matrix, route + [min_node])

def greedy(matrix, route=None):
    """
    It doesn't return an optimal route but its close to brute force answer.
    The benifit of the algorithm is relative speed and simplicity.
    """
    result = greedy_recursion(matrix, route or [0])
    return result, total_cost(m, result)

## Branch and Bound

Implementation was inspired by [this](http://galyautdinov.ru/post/zadacha-kommivoyazhera) post.

In [6]:
from random import randint
def random_matrix(n, max_distance=10):
    return [[ii == jj and float('inf') or randint(1, max_distance) for ii in range(n)] for jj in range(n)]

In [7]:
def copy_matrix(m):
    return [[ii for ii in jj] for jj in m]

In [8]:
def min_in(matrix, direction='row'):
    vector = []
    d = range(len(matrix))
    for ii in d:
        min_element = float('inf')
        for jj in d:
            # switch between row and column access
            item = direction == 'row' and matrix[ii][jj] or matrix[jj][ii]
            if min_element > item:
                min_element = item
        vector.append(min_element)

    return vector

In [9]:
def subtract(matrix, vector, direction='row'):
    d = range(len(matrix))
    for ii in d:
        if vector[ii] == float('inf'):
            continue
        for jj in d:
            if direction == 'row':
                matrix[ii][jj] -= vector[ii]
            else:
                matrix[jj][ii] -= vector[ii] 
    return matrix

In [10]:
def reduse(matrix):
    v1 = min_in(matrix)
    redused_rows = subtract(matrix, v1)
    v2 = min_in(redused_rows, 'column')
    return subtract(redused_rows, v2, 'column')

In [11]:
def find_optimal_segment(matrix):
    size = range(len(matrix))
    max_value = -1
    max_ii = max_jj = -1

    for ii in size:
        for jj in size:
            if matrix[ii][jj] != 0:
                continue

            # calculate max value for cells that have zeros
            min_in_row = min([e for pos, e in enumerate(matrix[ii]) if pos != jj])
            min_in_column = min([e[jj] for pos, e in enumerate(matrix) if pos != ii])
            current_max = min_in_row + min_in_column
            if max_value < current_max:
                max_value = current_max
                max_ii = ii
                max_jj = jj

    # close the route back
    matrix[max_jj][max_ii] = float('inf')

    # "remove" row and column
    for jj in size:
        matrix[max_ii][jj] = float('inf')
    for ii in size:
        matrix[ii][max_jj] = float('inf')

    return matrix, max_ii, max_jj


In [12]:
def branch_and_bound(matrix):
    # reduse modifies matrix in place
    # so we copy the original in order
    # to not mess it up
    m = copy_matrix(matrix)

    # initialize variables
    path_dict = {}
    distance = 0
    size = range(len(matrix))

    # find optimal pairs
    for i in size:
        redused = reduse(m)
        m, a, b = find_optimal_segment(redused)
        path_dict[a] = b
        distance += matrix[a][b]
    
    # arrange segments in walking order
    path = []
    for i in size:
        # start with 0 if it is a new path else reassign to second element
        a = path and b or i
        path.append(a)

        b = path_dict[a]

    return path, distance

## Comparing algorithms

In [13]:
m = random_matrix(5)
m

[[inf, 10, 8, 5, 4],
 [7, inf, 2, 3, 4],
 [7, 6, inf, 7, 6],
 [1, 4, 7, inf, 2],
 [10, 2, 7, 9, inf]]

In [14]:
branch_and_bound(m)

([0, 4, 1, 2, 3], 16)

In [15]:
brute_force(m)

([0, 4, 1, 2, 3], 16)

In [16]:
greedy(m)

([0, 4, 1, 2, 3], 16)

## Look ahead

In [55]:
test = random_matrix(20)

In [99]:
def sub_route_cost(matrix, route, loop=True):
    # we are going from node a to node b in each step
    a = list(route)
    b = a[1:] + [a[0]]

    # calculate the route cost
    cost = 0
    for ii, jj in zip(a, b):
        cost += matrix[ii][jj]
    if not loop:
        # exclude path back to start
        cost -= matrix[route[0]][route[-1]]
    return cost

def look_ahead(matrix, k):
    path = []
    cost = 0
    size = len(matrix)

    stop = 0
    while True:
        start = stop
        stop = start + k

        if stop > size - 1:
            stop = size - 1

        if start == stop:
            # Nowhere to go from here
            cost += matrix[path[0]][path[-1]]
            print('route back')
            break

        if stop - start == 1:
            # Only one possible route
            path += [stop]
            cost += matrix[start][stop]
            print('last step')
            continue

        current_min = float('inf')
        current_route = []
        for route in permutations(range(start, stop)):
            route_cost = sub_route_cost(matrix, route, loop=False)
            # check if we got new min route
            if route_cost < current_min:
                current_min = route_cost
                current_route = route

        sub_route = list(current_route)
        sub_cost = current_min
        
        path += sub_route
        cost += sub_cost

    return path, cost

In [105]:
look_ahead(test, 2)

last step
route back


([1, 0, 2, 3, 5, 4, 7, 6, 8, 9, 10, 11, 13, 12, 15, 14, 17, 16, 19], 55)

In [106]:
look_ahead(test, 3)

last step
route back


([1, 2, 0, 3, 4, 5, 8, 6, 7, 10, 9, 11, 13, 14, 12, 17, 15, 16, 19], 35)

In [107]:
look_ahead(test, 5)

route back


([1, 2, 0, 4, 3, 9, 6, 8, 5, 7, 13, 11, 10, 14, 12, 17, 18, 15, 16], 38)